<a href="https://colab.research.google.com/github/ttogle918/classify_instrument_by_CNN/blob/master/wav_to_df_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install librosa

In [1]:
from google.colab import drive

drive.mount('/content/drive')
path = '/content/drive/My Drive/Colab Notebooks/code_states/s4_project/'

wav_train_path = 'FSDKaggle2018.audio_train/'
wav_test_path = 'FSDKaggle2018.audio_test/'

Mounted at /content/drive


In [2]:
import librosa
import librosa.display
import numpy as np
import matplotlib.pyplot as plt
import itertools
import pandas as pd

In [13]:
df = pd.read_csv(path+'train.csv')

In [4]:
df.head()

,fname,label,manually_verified
0,00044347.wav,Hi-hat,0
1,001ca53d.wav,Saxophone,1
2,002d256b.wav,Trumpet,0
3,0033e230.wav,Glockenspiel,1
4,00353774.wav,Cello,1


In [8]:
df.label.unique()

array(['Hi-hat', 'Saxophone', 'Trumpet', 'Glockenspiel', 'Cello', 'Knock',
       'Gunshot_or_gunfire', 'Clarinet', 'Computer_keyboard',
       'Keys_jangling', 'Snare_drum', 'Writing', 'Laughter', 'Tearing',
       'Fart', 'Oboe', 'Flute', 'Cough', 'Telephone', 'Bark', 'Chime',
       'Bass_drum', 'Bus', 'Squeak', 'Scissors', 'Harmonica', 'Gong',
       'Microwave_oven', 'Burping_or_eructation', 'Double_bass',
       'Shatter', 'Fireworks', 'Tambourine', 'Cowbell', 'Electric_piano',
       'Meow', 'Drawer_open_or_close', 'Applause', 'Acoustic_guitar',
       'Violin_or_fiddle', 'Finger_snapping'], dtype=object)

- MIDI에 없는 소리 : Hi-hat, Gunshot_or_gunfire, Knock, Computer_keyboard, Keys_jangling, Writing, Laughter, Tearing, Fart, Cough, Bark, Chime,  Bus, Squeak, Scissors, Gong, Microwave_oven, Burping_or_eructation, Shatter, Fireworks, Tambourine, Cowbell, Meow, Drawer_open_or_close, 
- Clarinet : 72
- Saxophone : 소프라노 색소폰? 65
- Trumpet : 57
- Glockenspiel : 10
- Cello : 43
- Snare_drum = side drum : ? steel drums : 115
- Oboe : 69
- Flute : 74
- Telephone : 125
- Bass_drum : 115 ( steel drums )
- Harmonica : 23
- Double_bass : 33?
- Electric_piano :5, 6
- Applause : 127
- Acoustic_guitar : 25 26 
- Violin_or_fiddle : 41, 111

'Clarinet','Saxophone', 'Trumpet', 'Glockenspiel', 'Cello', 'Snare_drum', 'Oboe', 'Flute', 'Telephone', 'Bass_drum', 'Harmonica', 'Double_bass', 'Electric_piano', 'Applause', 'Acoustic_guitar', 'Violin_or_fiddle'

In [14]:
df = df[df.label.isin(['Clarinet','Saxophone', 'Trumpet', 'Glockenspiel', 'Cello', 'Snare_drum', 'Oboe', 'Flute', 'Telephone', 'Bass_drum', 'Harmonica', 'Double_bass', 'Electric_piano', 'Applause', 'Acoustic_guitar', 'Violin_or_fiddle'])]

In [15]:
df.shape

(4128, 3)

In [12]:
df.head()

0    False
1     True
2     True
3     True
4     True
Name: label, dtype: bool

In [16]:
from tqdm import tqdm

audio_data = list()

for i in tqdm(range(df.shape[0])):
    audio_data.append(librosa.load(path+wav_train_path+df['fname'].iloc[i]))

audio_data = np.array(audio_data)

100%|██████████| 4128/4128 [1:14:29<00:00,  1.08s/it]
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  


In [17]:
# df에 추가
df['audio_waves'] = audio_data[:,0]
df['samplerate'] = audio_data[:,1]
df.head()

,fname,label,manually_verified,audio_waves,samplerate
1,001ca53d.wav,Saxophone,1,"[-0.0007303721, -0.0010996412, -0.0010551845, ...",22050
2,002d256b.wav,Trumpet,0,"[-1.8233735e-06, -3.217056e-05, -4.674272e-05,...",22050
3,0033e230.wav,Glockenspiel,1,"[-0.00067174545, 0.0006391616, -0.0013553092, ...",22050
4,00353774.wav,Cello,1,"[-0.003822653, -0.0053708917, -0.0044372473, -...",22050
5,003b91e8.wav,Cello,0,"[0.0031921505, 0.0048865937, 0.004513402, 0.00...",22050


In [18]:
bit_lengths = list()    # 비트 길이

for i in range(df.shape[0]):
    bit_lengths.append(len(df['audio_waves'].iloc[i]))
bit_lengths = np.array(bit_lengths)
df['bit_lengths'] = bit_lengths
df['seconds_length'] = df['bit_lengths']/df['samplerate']   # 소리의 길이(wav 파일 실행 시간)

df.head()

,fname,label,manually_verified,audio_waves,samplerate,bit_lengths,seconds_length
1,001ca53d.wav,Saxophone,1,"[-0.0007303721, -0.0010996412, -0.0010551845, ...",22050,227556,10.32
2,002d256b.wav,Trumpet,0,"[-1.8233735e-06, -3.217056e-05, -4.674272e-05,...",22050,9702,0.44
3,0033e230.wav,Glockenspiel,1,"[-0.00067174545, 0.0006391616, -0.0013553092, ...",22050,176400,8
4,00353774.wav,Cello,1,"[-0.003822653, -0.0053708917, -0.0044372473, -...",22050,99666,4.52
5,003b91e8.wav,Cello,0,"[0.0031921505, 0.0048865937, 0.004513402, 0.00...",22050,292824,13.28


In [21]:
# 2초 이상의 data만 남기기 ()
df = df[df['seconds_length'] >= 2.0]

min_bits = np.min(df['bit_lengths'])
print(min_bits)   # 44100(비트)

min_seconds = np.min(df['seconds_length'])
print(min_seconds)    # 2.0(초)

44100
2.0


In [23]:
import pickle
with open('audio_df_cleaned.pickle', 'wb') as f:
    pickle.dump(df, f)